# Análise de Sentimentos - Exportação do Modelo

Este notebook faz o treinando de machine learning de um modelo de análise de sentimentos, utilizando o dataset do Buscapé fornecido (confidencial), de forma que:
- Carrega e estrutura os dados do dataset em dataframe
- Constrói o modelo de machine learning utilizando TF-IDF e Logist Regression
- Exporta o modelo em dump pickle para ser utilizado na biblioteca nlputils

## Configurações Iniciais

Primeiramente, fazer o ajuste no path do Python para a raíz do projeto:

In [1]:
import os
os.chdir('..')

--------------------------------------

## Processamento do Dataset

Carregar a bliblioteca de normalização de texto e de dump de dados:

In [2]:
from nlputils.lexical.normalizer import Normalizer
import pickle

Instanciar um objeto para fazer as normalizações necessárias:

In [3]:
normalizer = Normalizer()

Implementar uma rotina de pré-processamento a ser aplicada em todo o dataset antes de construir o modelo e uma para auxiliar o carregamento dos dados, evitando replicação de código.

A função ```preprocessing``` recebe um texto (string) de entrada e o deixa normalizado, conforme procedimentos de nível lexical, e retorna o próprio texto (string) mais compacto.

In [4]:
def preprocessing(text):
    text = normalizer.to_lowercase(text)
    text = normalizer.remove_ponctuations(text)
    tokens = normalizer.tokenize_words(text)
    tokens = normalizer.remove_stopwords(tokens)
    return ' '.join(tokens)

A função ```dumpvar``` salva em dados binário todo o valor de uma variável na memória. Basta fornecer o ponteiro do identificador da variável e o caminho onde será salvo o dump.

In [5]:
def dumpvar(var, path):
    path_itens = path.split('/')
    path_dir = '/'.join(path_itens[:-1]) + '/'
    
    if not os.path.isdir(path_dir):
        os.makedirs(path_dir)
    
    with open(path, 'wb') as fp:
        pickle.dump(var, fp)

A função ```loadvar``` retorna o valor de uma dump de memória salvo em disco. Basta o caminho onde está salvo o dump e atribuir em uma variável.

In [6]:
def loadvar(path):
    with open(path, 'rb') as fp:
        return pickle.load(fp)

---------------------------------

## Carregando o Dataframe

Carregando a partir do dump gerado no notebook ```4_Sentiment_Analysis_Testing```:

In [7]:
dataframe = loadvar('data/dump/dataframe.pickle')

----------------------------

## Construindo o Modelo de Análise de Sentimentos

Importar os módulos de TF-IDFe  Logistic Regression:

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

Importar biblioteca para cronometrar tempo dos algoritmos:

In [9]:
import time

Separa os dados para serem usados nos modelos.

In [10]:
train_reviews = dataframe['review'].values.tolist()
train_classes = dataframe['polarity'].values.tolist()

### Tranformer TF-IDF

Instancia um objeto para o transformador TF-IDF.

In [11]:
transformer = TfidfVectorizer()

Ajusta (treina) o modelo de transformação TF-IDF a partir do trainset.

In [12]:
start_time = time.perf_counter()

transformer.fit(train_reviews)

end_time = time.perf_counter()
print('TEMPO DE EXECUÇÃO: {:.2f} segundos'.format(end_time - start_time))

TEMPO DE EXECUÇÃO: 2.76 segundos


Transforma os dados em string do dataset para vetores TF-IDF.

In [13]:
X = transformer.transform(train_reviews)

Dump da variável, para ser usado na biblioteca nlputils:

In [14]:
dumpvar(transformer, 'data/dump/transformer.pickle')

In [ ]:
# transformer = loadvar('data/dump/transformer.pickle')

### Classificador Logistic Regression

Instancia um objeto para o modelo do classificador.

> **ATENÇÃO:** Ajuste o parâmetro ```n_jobs``` para a quantidade de _cores físicos_ do seu CPU.

In [15]:
classifier = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=200, n_jobs=4)
# classifier = LogisticRegression(solver='warn', multi_class='warn')

Treinando o modelo de classificador de sentimentos com Logistic Regression.

In [16]:
start_time = time.perf_counter()

classifier.fit(X, train_classes)

end_time = time.perf_counter()
print('TEMPO DE EXECUÇÃO: {:.2f} segundos'.format(end_time - start_time))

TEMPO DE EXECUÇÃO: 49.02 segundos


Dump da variável, para ser usado na biblioteca nlputils:

In [17]:
dumpvar(classifier, 'data/dump/classifier.pickle')

In [ ]:
# classifier = loadvar('data/dump/classifier.pickle')

------------------------

Pequeno teste do modelo final:

In [25]:
sentence = "Produto excelente!!!"

In [26]:
normalized = preprocessing(sentence)
instance = transformer.transform([normalized])
print('Score: ', classifier.predict(instance)[0])

Score:  5
